# Task 1: Word Embeddings (10 points)

This notebook will guide you through all steps necessary to train a word2vec model (Detailed description in the PDF).

## Imports

This code block is reserved for your imports. 

You are free to use the following packages: 

(List of packages)

In [1]:
# Imports
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import torch as torch
from torch import nn
#nltk.download('stopwords')
#nltk.download("punkt")
from nltk.tokenize import word_tokenize
import math as mt
print("Done")

Done


# 1.1 Get the data (0.5 points)

The Hindi portion HASOC corpus from [github.io](https://hasocfire.github.io/hasoc/2019/dataset.html) is already available in the repo, at data/hindi_hatespeech.tsv . Load it into a data structure of your choice. Then, split off a small part of the corpus as a development set (~100 data points).

If you are using Colab the first two lines will let you upload folders or files from your local file system.

In [2]:
#TODO: implement!

#from google.colab import files
#uploaded = files.upload()

data = pd.read_csv('data/hindi_hatespeech.tsv', sep='\t')
data_development = data[:100]
type(data_development['task_1'])

print("Done")

Done


## 1.2 Data preparation (0.5 + 0.5 points)

* Prepare the data by removing everything that does not contain information. 
User names (starting with '@') and punctuation symbols clearly do not convey information, but we also want to get rid of so-called [stopwords](https://en.wikipedia.org/wiki/Stop_word), i. e. words that have little to no semantic content (and, but, yes, the...). Hindi stopwords can be found [here](https://github.com/stopwords-iso/stopwords-hi/blob/master/stopwords-hi.txt) Then, standardize the spelling by lowercasing all words.
Do this for the development section of the corpus for now.

* What about hashtags (starting with '#') and emojis? Should they be removed too? Justify your answer in the report, and explain how you accounted for this in your implementation.

In [3]:
#TODO: implement!
hindi_stopword_file = open('data/stopwords-hi.txt', encoding="utf8")

sw_list = ['#', '?', '!', ';', ',', ':', "\'", '-', '=', '(', ')', '[', ']' , '{', '}', '"', '*', '@', '  ', '\\', '/', '..', '...', '....', '%'
          ,'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '\t']
sw_list_string = ''
for i in sw_list:
    sw_list_string+=i
hindi_stopwords = []
for x in hindi_stopword_file:
    hindi_stopwords.append(x.rstrip())

hindi_stopwords.extend(sw_list)
sentences = []
# text = data_development['text'].iloc[0] + ' @Zalman'
for text in data_development['text']:
    text_array = text.split(' ')
    # text_tokens =word_tokenize(text) #data_development["text"].apply(nltk.word_tokenize)
#     print(text_array)
    new_array = []
    for j in text_array:
        if '@' not in j and len(j) < 20:
            for char in sw_list:
                j = j.replace(char, '')
            new_array.append(j)
    sentences.append(' '.join(new_array))
                
# tokens_without_sw = [word for word in text_tokens if not word in hindi_stopwords]
# filtered_sentence = (" ").join(tokens_without_sw)
# #text_tokens
# filtere_sentence
# print(sentences)

print("Done")

Done


## 1.3 Build the vocabulary (0.5 + 0.5 points)

The input to the first layer of word2vec is an one-hot encoding of the current word. The output od the model is then compared to a numeric class label of the words within the size of the skip-gram window. Now

* Compile a list of all words in the development section of your corpus and save it in a variable ```V```.

In [4]:
#TODO: implement!
temp_unique = [] # For unique words
temp_nounique = []
for j in sentences:
    temp2 = j.split(' ')
    for k in temp2:
        if k not in temp_unique:
            temp_unique.append(k)
        temp_nounique.append(k)
V = temp_unique
non_unique = temp_nounique
print(len(non_unique))
print(len(V))
print(sentences[0])

2806
1313
बांग्लादेश की शानदार वापसी भारत को  रन पर रोका INDvBAN CWC


* Then, write a function ```word_to_one_hot``` that returns a one-hot encoding of an arbitrary word in the vocabulary. The size of the one-hot encoding should be ```len(v)```.

In [5]:
#TODO: implement!
def word_to_one_hot(word):
    if word in V:
        index = V.index(word)
        encoding = np.zeros(len(V))
        encoding[index]= 1
        return encoding
    return False
#   pass

## 1.4 Subsampling (0.5 points)

The probability to keep a word in a context is given by:

$P_{keep}(w_i) = \Big(\sqrt{\frac{z(w_i)}{0.001}}+1\Big) \cdot \frac{0.001}{z(w_i)}$

Where $z(w_i)$ is the relative frequency of the word $w_i$ in the corpus. Now,
* Calculate word frequencies
* Define a function ```sampling_prob``` that takes a word (string) as input and returns the probabiliy to **keep** the word in a context.

In [6]:
#TODO: implement!
def sampling_prob(word):
    z_wi = non_unique.count(word) / len(non_unique)
    p_wi = (mt.sqrt(z_wi / 0.001) + 1) * (0.001 / z_wi)
    return p_wi
#   pass

In [7]:
sampling_prob('बांग्लादेश')
sample_onehot = word_to_one_hot('बांग्लादेश')
print(sample_onehot.shape)
print(len(sentences))
print(sample_onehot)

(1313,)
100
[1. 0. 0. ... 0. 0. 0.]


# 1.5 Skip-Grams (1 point)

Now that you have the vocabulary and one-hot encodings at hand, you can start to do the actual work. The skip gram model requires training data of the shape ```(current_word, context)```, with ```context``` being the words before and/or after ```current_word``` within ```window_size```. 

* Have closer look on the original paper. If you feel to understand how skip-gram works, implement a function ```get_target_context``` that takes a sentence as input and [yield](https://docs.python.org/3.9/reference/simple_stmts.html#the-yield-statement)s a ```(current_word, context)```.

* Use your ```sampling_prob``` function to drop words from contexts as you sample them. 

In [8]:
#TODO: implement!
def get_target_context(sentence):
    sentence_split = sentence.split(' ')
    for word in sentence_split:
        context = []
        index = sentence_split.index(word)
        for i in range(len(sentence_split)):
            if(i != index):
                context.append(word)
        yield(word, context)

hello = get_target_context("Sohaib and Salman")
for val in hello:
    print(val)

('Sohaib', ['Sohaib', 'Sohaib'])
('and', ['and', 'and'])
('Salman', ['Salman', 'Salman'])


# 1.6 Hyperparameters (0.5 points)

According to the word2vec paper, what would be a good choice for the following hyperparameters? 

* Embedding dimension
* Window size

Initialize them in a dictionary or as independent variables in the code block below. 

In [9]:
# Set hyperparameters
# window_size = from 1 to 10
# window_size = from 500 to 1000 (640 used in paper)
window_size = 4
embedding_size = 10

# More hyperparameters
learning_rate = 0.05
epochs = 100

# 1.7 Pytorch Module (0.5 + 0.5 + 0.5 points)

Pytorch provides a wrapper for your fancy and super-complex models: [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). The code block below contains a skeleton for such a wrapper. Now,

* Initialize the two weight matrices of word2vec as fields of the class.

* Override the ```forward``` method of this class. It should take a one-hot encoding as input, perform the matrix multiplications, and finally apply a log softmax on the output layer.

* Initialize the model and save its weights in a variable. The Pytorch documentation will tell you how to do that.

In [10]:
# Create model 

class Word2Vec(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = torch.nn.Parameter(torch.randn((len(V), embedding_size)), requires_grad = True)
        self.w2 = torch.nn.Parameter(torch.randn((embedding_size, len(V))), requires_grad = True)


    def forward(self, one_hot):
        output_layer1 = torch.matmul(one_hot, self.w1.type(torch.DoubleTensor))
        matrix_mul = torch.matmul(output_layer1, self.w2.type(torch.DoubleTensor))
        matrix_mul = matrix_mul.cpu().detach().numpy()
        e_x = np.exp(matrix_mul - np.max(matrix_mul))
        log_softmax = np.log(e_x / e_x.sum())
        return torch.tensor(log_softmax)
net = Word2Vec()
#     pass

print("Done")
w = []
for param in net.parameters():
    w.append(param.data)
# w = np.array(w)
# print(w[0].shape)

Done


# 1.8 Loss function and optimizer (0.5 points)

Initialize variables with [optimizer](https://pytorch.org/docs/stable/optim.html#module-torch.optim) and loss function. You can take what is used in the word2vec paper, but you can use alternative optimizers/loss functions if you explain your choice in the report.

In [11]:
# Define optimizer and loss
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.BCEWithLogitsLoss()

print("Done")

Done


# 1.9 Training the model (3 points)

As everything is prepared, implement a training loop that performs several passes of the data set through the model. You are free to do this as you please, but your code should:

* Load the weights saved in 1.6 at the start of every execution of the code block
* Print the accumulated loss at least after every epoch (the accumulate loss should be reset after every epoch)
* Define a criterion for the training procedure to terminate if a certain loss value is reached. You can find the threshold by observing the loss for the development set.

You can play around with the number of epochs and the learning rate.

In [12]:

# Define train procedure

# load initial weights
# count = 0
def train():
    net = Word2Vec()
    print("Training started")
    for e in range(epochs):
        count = 0
        for i in sentences:
            words_in_sentence = i.split(' ')
            for j in range(len(words_in_sentence)):
                input_vec = word_to_one_hot(words_in_sentence[j])
                input_vec = input_vec.reshape((1,len(V)))
#                 print('input_vec', input_vec.shape)
                upper_bound = j + window_size + 1
                lower_bound = j - window_size
                for k in range(lower_bound, upper_bound):
                    if k>=0 and k<len(words_in_sentence) and k!=j:
#                         count += 1
                        output_vec = word_to_one_hot(words_in_sentence[k])
                        output_vec = output_vec.reshape((1,len(V)))
                        y_pred_torch = net(torch.tensor(input_vec))
                        y_pred_torch.requires_grad = True
                        
#                         print(y_pred_torch.requires_grad)
                        loss = criterion(y_pred_torch, torch.tensor(output_vec))
                        if count == 0:
                            print(loss.item())
                        count = 1
                        # print(t, loss.item())
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
#         print('Main: ',e, loss.item())

train()

print("Training finished")

Training started
0.010108209247961911
0.010108209247961911
0.010108209247961911
0.010108209247961911


KeyboardInterrupt: 

# 1.10 Train on the full dataset (0.5 points)

Now, go back to 1.1 and remove the restriction on the number of sentences in your corpus. Then, reexecute code blocks 1.2, 1.3 and 1.6 (or those relevant if you created additional ones). 

* Then, retrain your model on the complete dataset.

* Now, the input weights of the model contain the desired word embeddings! Save them together with the corresponding vocabulary items (Pytorch provides a nice [functionality](https://pytorch.org/tutorials/beginner/saving_loading_models.html) for this).